In [2]:
with open( "/home/soren/Documents/oneshot/train.pickle","r") as f:
    (X,y,categories) = pickle.load(f)

with open( "/home/soren/Documents/oneshot/val.pickle","r") as f:
    (Xval,yval,categoriesv) = pickle.load(f)
print(Xval.shape)
a=siamese_net.predict([X[1,1,:,:].reshape(1,105,105,1),X[7,1,:,:].reshape(1,105,105,1) ])
print(a)
print("?")

(659, 20, 105, 105)
[[ 0.498815]]
?


In [3]:
from time import time



class Siamese_Loader:
    def __init__(self,Xval,Xtrain):
        self.Xval = Xval
        self.Xtrain = Xtrain
        self.n_classes,self.n_examples,self.w,self.h = Xtrain.shape
    
    def get_batch(self,X,n,same=False):
        """sample n pairs of training images with the same class if same"""
        categories = rng.choice(self.n_classes,size=(n,),replace=False)
        pairs=[np.zeros((n, self.h, self.w)),np.zeros((n, self.h, self.w))]
        for i in range(n):
            category = categories[i]
            idx_1 = rng.randint(0,self.n_examples)
            pairs[0][i] = X[category,idx_1,:,:]
            if same == True:
                idx_2 = rng.randint(0,n_examples)
                category_2 = category
            else:
                idx_2 = rng.randint(0,n_examples)
                category_2 = rng.choice(range(0,category)+range(category+1,n_classes))
            pairs[1][i] = X[category_2,idx_2,:,:]
        return pairs

    def make_oneshot_task(X,n):
        """returns 2  n x h x w tensors to input"""
        categories = rng.choice(self.n_classes,size=(n,),replace=False)
        true_category = categories[0]
        pairs=[np.zeros((n, self.h, self.w)),np.zeros((n, self.h, self.w))]
        ex1, ex2 = rng.choice(n_examples,replace=False,size=(2,))
        test_image = X[true_category,ex1,:,:]
        for i in range(1,n):
            rand_idx = rng.randint(0,n_examples)
            pairs[0][i] = test_image
            pairs[1][i] = X[other_category,rand_idx,:,:]
        pairs[0][n] = test_image
        pairs[0][1] = X[true_category,ex2]
        targets = np.zeros((n,))
        targets[0] = 1

t=time()
pairs = pick(X,64,same=True)
#v=X[0:128,1,:,:]
dt = time()-t
print("took {} seconds to get batch".format(dt))
plt.matshow(pairs[0][3])
plt.matshow(pairs[1][3])


plt.show()


NameError: name 'pick' is not defined

In [ ]:

n=64
t = time()
(a,targets)=get_batch(X,y,n)
dt = time() - t
print("getting a batch of size {} took {}".format(n,dt))

for i in range(20000):
    (inputs,targets)=get_batch(X,y,64)
    loss=siamese_net.train_on_batch(inputs,targets)
    if i % 170 == 0:
        val_input,val_target=get_batch(Xval,yval,32)
        val_loss=siamese_net.evaluate(val_input,val_target)
        print("iteration: {}, training loss: {}, val loss: {}".format(i,loss, val_loss))
    

(inputs,targets)=get_batch(Xval,yval,64)



plt.matshow(np.hstack([siamese_net.predict(inputs), targets.reshape(64,1)])[16:48],cmap='gray')
plt.show()
targets-siamese_net.predict(inputs)

In [ ]:
preds=siamese_net.predict(inputs)
#print(np.where(preds==preds.max()))
#print(inputs[0][58].shape)
for idx in range(3):
    print("probability not same:{}".format(preds[idx]))
    plt.matshow(inputs[0][idx].reshape(105,105),cmap='gray')
    plt.matshow(inputs[1][idx].reshape(105,105),cmap='gray')

    plt.show()
#plt.matshow(inputs[1][58])
